In [9]:
from mpi4py import MPI
import json

# read parameters
params =

comm = MPI.COMM_WORLD
mode = MPI.MODE_RDONLY
fh = MPI.File.Open(comm, "data-big.txt", mode)

# set read size parameter
file_size = fh.Get_size()
num_process = comm.Get_size()
rank = comm.Get_rank()
chunk_size = int(file_size / num_process)
start = rank * chunk_size + 1

# read all
buf = bytearray(chunk_size)
fh.Iread_at(start, buf)

raw = buf.decode()

print("Process", rank, "finish reading. Total size is", len(raw), "bytes.")

Exception: Invalid argument, error stack:
MPI_FILE_IREAD_AT(100): Invalid offset argument

In [22]:
import json
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
fh = MPI.File.Open(comm, "config", MPI.MODE_RDONLY)
file_size = fh.Get_size()
buf = bytearray(file_size)
fh.Iread_at(0, buf)
params = json.loads(buf.decode())

# with open("config", "r") as f:
#     params = json.load(f)

print("Process", rank, "finish", "Answer: ", params)

Process 0 finish Answer:  {'chunk_size': 100000}


In [2]:
ts = [(x[0], float(x[1]), int(x[2])) for row in raw.split("\n") for x in [row.split(",")] if len(x) == 3]
print(len(ts))
print(ts[:5])
print(ts[-5:])

KeyboardInterrupt: 

In [4]:
raw[:100]

'20140804:10:00:00.574914,1173.56,471577\n20140804:10:00:00.898688,1251.60,445361\n20140804:10:00:00.94'

In [133]:
# find out the longest misplacement

tmp = [x[0] for x in ts]
rank = np.argsort(tmp)
rev = np.maximum(rank - np.arange(len(tmp)), 0)
cdf = np.sort(rev)

print(cdf[99000]) # 99% quantile is 871
print(cdf[99900]) # 99.9% quantile is 1085
print(cdf[-1]) # 100% quantile is 99882
print(np.sum(cdf>0)/len(cdf)) # 50% of records are misplaced

871
1085
99882
0.49394


In [136]:
# duplicates - record, time, (price, vol)
# a paper about filtering tick data
# erratic price on thin volume
# t2 from (t1, t2, t3) is out of wreck. use brownian bridge?

In [8]:
from mpi4py import MPI
import numpy as np
import json
import datetime
import os
from operator import itemgetter

def log(fh, rank, code, msg):
    # code: 0 - Info, 1 - Warning, 2 - Error
    codex = {0: "INFO", 1: "WARNING", 2: "ERROR"}
    msg = "[" + str(rank) + "][" + codex[code] + "]" + " (" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") \
          + "): " + msg
    fh.Write_shared(bytearray(msg.encode()))

if __name__ == "__main__":

    # start MPI and get basic info
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    num_nodes = comm.Get_size()

    # reset log file
    if rank == 0:
        try:
            os.remove("log")
        except:
            pass

    flog = MPI.File.Open(comm, "log", MPI.MODE_WRONLY | MPI.MODE_CREATE)

    # read parameters
    try:
        with open("config", "r") as f:
            params = json.load(f)

        chunk_size = params["chunk_size"]
        chunk_buf = params["chunk_buf"]
        overlap_size = params["overlap_size"]

        log(flog, rank, 0, "Loaded in parameters")

    except:
        log(flog, rank, 2, "Fail to load in parameters. Set to defaults")

        chunk_size = 10000
        chunk_buf = 50
        overlap_size = 1000



    # read file in chunks
    fh = MPI.File.Open(comm, "data-small.txt", MPI.MODE_RDONLY)
    fsig = MPI.File.Open(comm, "signal.txt", MPI.MODE_WRONLY | MPI.MODE_CREATE)
    fnoi = MPI.File.Open(comm, "noise.txt", MPI.MODE_WRONLY | MPI.MODE_CREATE)

    file_size = fh.Get_size()
    block_size = file_size / num_nodes
    block_offset = rank * block_size

    num_chunks = int(block_size // chunk_size)
    overlap = []
    for iter in range(1):
        buf = bytearray(chunk_size + chunk_buf)
        fh.Read_at(block_offset + iter * chunk_size, buf)

        # discard everything before the first "\n"
        rows = buf[:chunk_size].decode().split("\n")[1:]

        # get remaining part from the remainder
        extra = buf[chunk_size:].decode().split("\n")[0]
        # if last byte is "\n", take one more record from the remainder
        if buf[chunk_size - 1] == 10:
            rows.append(extra)
        else:
            # append the remaining part of last row from the remainder
            rows[-1] += extra

        parsed = overlap + [(x[0], float(x[1]), int(x[2])) for row in rows for x in [row.split(",")]]
        # sort by time
        sorted_ix = np.argsort([x[0] for x in parsed])

        signal = []
        noise = []
        for i, j in zip(sorted_ix[:-overlap_size], sorted_ix[1:-(overlap_size-1)]):
            # filter by duplicates, price anomaly and negative volume
            if parsed[i] == parsed[j] or parsed[i][1] <= 0 or parsed[i][1] > 50000 or parsed[i][2] <= 0:
                noise.append(i)
            else:
                signal.append(i)

        overlap = [parsed[ix] for ix in sorted_ix[-overlap_size:]]
        
        # output result
        # fsig.Write_shared(bytearray("\n".join([parsed[ix] for ix in signal]).encode()))
        # fnoi.Write_shared(bytearray("\n".join([parsed[ix] for ix in noise]).encode()))

    # close files
    flog.Close()
    fh.Close()
    fsig.Close()
    fnoi.Close()

In [43]:
with open("output/noise.txt", "r") as f:
    raw = f.read()

to_compare = [row for row in raw.split("\n") if row]

with open("output/noise_benchmark.txt", "r") as f:
    raw = f.read()

bench = raw.split("\n")

In [44]:
A = [row for row in to_compare if row not in bench]
print(A)

[]


In [45]:
B = [row for row in bench if row not in to_compare]
print(B)

['']


In [55]:
with open("data/data-big.txt", "r") as f:
    f.seek(40000000)
    raw = f.read(500050)

raw[-30:]

'0799,1200.10,390294\n20140804:1'